In [2]:
import tensorflow as tf
# import plotly as px
import pandas as pd
# import matplotlib.pyplot as plt
import numpy as np
# import plotly.express as px
import copy
# from Modules.Technical_indicators import ATR, BB, RSI, ADX, MACD
# from plotly.subplots import make_subplots
# import plotly.graph_objs as go
# from Modules.Plotting import Compare_Plot, Compare_Plot_bar
import warnings
warnings.filterwarnings("ignore")



In [13]:
df = pd.read_csv(r'D:\Time_series_project\BTC-USD.csv')
df['Date'] = pd.to_datetime(df['Date'])
df_time = df.set_index('Date')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3503 entries, 0 to 3502
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       3503 non-null   datetime64[ns]
 1   Open       3503 non-null   float64       
 2   High       3503 non-null   float64       
 3   Low        3503 non-null   float64       
 4   Close      3503 non-null   float64       
 5   Adj Close  3503 non-null   float64       
 6   Volume     3503 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 191.7 KB


In [14]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date',inplace = True)
data = copy.deepcopy(df)
df.dropna(inplace = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3503 entries, 2014-09-17 to 2024-04-19
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       3503 non-null   float64
 1   High       3503 non-null   float64
 2   Low        3503 non-null   float64
 3   Close      3503 non-null   float64
 4   Adj Close  3503 non-null   float64
 5   Volume     3503 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 191.6 KB


In [16]:
df = df.drop(columns = ['Open','High','Low','Close','Volume'],axis = 1)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3503 entries, 2014-09-17 to 2024-04-19
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Adj Close  3503 non-null   float64
dtypes: float64(1)
memory usage: 54.7 KB


In [17]:
labels_list = []
feature_list = []
window_size = 30
horizon = 1
feature_array = np.array(df)
for i in range(0, len(df['Adj Close'])-(window_size+horizon-1)):
    feature_list.append(np.array(feature_array[i:i+window_size]))
    labels_list.append(np.array(df['Adj Close'][i+window_size:i+window_size+horizon]))

# Convert the list of arrays to a numpy array
labels = np.array(labels_list)
feature = np.array(feature_list)

labels  = tf.convert_to_tensor(labels)
feature = tf.convert_to_tensor(feature)

In [18]:
labels.shape, feature.shape

(TensorShape([3473, 1]), TensorShape([3473, 30, 1]))

In [21]:
batch_size = 32
train_text_dataset = tf.data.Dataset.from_tensor_slices(feature)
train_labels_dataset = tf.data.Dataset.from_tensor_slices(labels)
train_dataset = tf.data.Dataset.zip((train_text_dataset, train_labels_dataset))
train_dataset = train_dataset.shuffle(buffer_size=len(feature)).batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [105]:
from tensorflow.keras.callbacks import EarlyStopping
# callback = EarlyStopping(
#     monitor="loss",#quantity to monitor
#     min_delta=0.0001,#Minimum change in the monnitored quantity as an improvement
#     patience=20,#No of epochs with no improvement
#     verbose=1,
#     mode="auto",
#     restore_best_weights=True
# )

In [106]:
# import datetime
# from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

# def create_callbacks(dir_name, experiment_name):
#     """
#     Creates callbacks for training.

#     Args:
#     - dir_name: target directory to store TensorBoard log files and model checkpoints
#     - experiment_name: name of experiment directory

#     Returns:
#     - callbacks: list of callbacks
#     """
#     log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#     print(f"Saving TensorBoard log files to: {log_dir}")

#     # TensorBoard callback for logging
#     tensorboard_callback = TensorBoard(log_dir=log_dir)

#     # ModelCheckpoint callback for saving the best model based on training loss
#     model_checkpoint_callback = ModelCheckpoint(filepath=f"{dir_name}/{experiment_name}/best_model.h5",
#                                                 monitor='loss',  # Monitor training loss
#                                                 save_best_only=True,
#                                                 verbose=1)

#     return [tensorboard_callback, model_checkpoint_callback]


### N_BEATS custom layers

In [8]:
# class NBeatsBlock(tf.keras.layers.Layer):
#     def __init__(self,
#                 input_size:int,
#                 theta_size:int,
#                 horizon:int,
#                 n_neurons:int,
#                 n_layers:int,
#                 **kwargs):

#         super().__init__(**kwargs)

#         # Initialize attributes based on input parameters
#         self.input_size = input_size  # Size of the input time series
#         self.theta_size = theta_size  # Size of the theta vector
#         self.horizon = horizon  # Forecasting horizon
#         self.n_neurons = n_neurons  # Number of neurons in each hidden layer
#         self.n_layers = n_layers  # Number of hidden layers

#         # Define hidden layers with ReLU activation
#         self.hidden = [tf.keras.layers.Dense(n_neurons, activation="relu") for _ in range(n_layers)]

#         # Define theta layer with linear activation
#         self.theta_layer = tf.keras.layers.Dense(theta_size, activation="linear", name="theta")

#     def call(self, inputs):
#         x = inputs  # Input tensor
#         # Pass input through each hidden layer sequentially
#         for layer in self.hidden:
#             x = layer(x)
        
#         # Compute theta vector by passing output of last hidden layer through theta layer
#         theta = self.theta_layer(x)

#         # Split theta vector into backcast and forecast components
#         backcast, forecast = theta[:, :self.input_size], theta[:, -self.horizon:]

#         # Return backcast and forecast predictions
#         return backcast, forecast


In [26]:
# N_EPOCHS = 1000
# N_NEURONS = 512
# N_LAYERS  = 4
# N_STACKS = 30

# input_size = window_size * horizon
# theta_size = window_size + horizon

# input_size, theta_size

(30, 31)

In [27]:
# window_size = 30
# horizon = 1

# class NBeatsBlock(tf.keras.layers.Layer):
#     def __init__(self,
#                 input_size:int,
#                 theta_size:int,
#                 horizon:int,
#                 n_neurons:int,
#                 n_layers:int,
#                 **kwargs):

#         super().__init__(**kwargs)

#         # Initialize attributes based on input parameters
#         self.input_size = input_size  # Size of the input time series
#         self.theta_size = theta_size  # Size of the theta vector
#         self.horizon = horizon  # Forecasting horizon
#         self.n_neurons = n_neurons  # Number of neurons in each hidden layer
#         self.n_layers = n_layers  # Number of hidden layers

#         # Define hidden layers with ReLU activation
#         self.hidden = [tf.keras.layers.Dense(n_neurons, activation="relu") for _ in range(n_layers)]

#         # Define theta layer with linear activation
#         self.theta_layer = tf.keras.layers.Dense(theta_size, activation="linear", name="theta")

#     def call(self, inputs):
#         x = inputs  # Input tensor
#         # Pass input through each hidden layer sequentially
#         for layer in self.hidden:
#             x = layer(x)
        
#         # Compute theta vector by passing output of last hidden layer through theta layer
#         theta = self.theta_layer(x)

#         # Split theta vector into backcast and forecast components
#         backcast, forecast = theta[:, :self.input_size], theta[:, -self.horizon:]

#         # Return backcast and forecast predictions
#         return backcast, forecast



# N_EPOCHS = 1000
# N_NEURONS = 512
# N_LAYERS  = 4
# N_STACKS = 30

# input_size = window_size * horizon
# theta_size = window_size + horizon

# from tensorflow.keras import layers 
# tf.random.set_seed(42)

# # Initialize the first N-Beats block layer
# nbeat_block_layer = NBeatsBlock(
#                                 input_size=input_size,
#                                 theta_size=theta_size,
#                                 horizon=horizon,
#                                 n_neurons=N_NEURONS,
#                                 n_layers=N_LAYERS,
#                                 name='Initial_block'
#                                 )

# # Define input layer for the stacked model
# stack_input = layers.Input(shape=(input_size), name='stack_input')

# # Apply the initial N-Beats block layer to the input layer
# backcast, forecast = nbeat_block_layer(stack_input)

# # Calculate residuals by subtracting the backcast predictions from the input
# residuals = layers.subtract([stack_input, backcast], name="subtract_00")

# # Iterate over the remaining N-Beats blocks to create a stacked architecture
# for i, _ in enumerate(range(N_STACKS - 1)):
#     # Apply N-Beats block to residuals from the previous block
#     backcast, block_forecast = NBeatsBlock(
#                                 input_size=input_size,
#                                 theta_size=theta_size,
#                                 horizon=horizon,
#                                 n_neurons=N_NEURONS,
#                                 n_layers=N_LAYERS,
#                                 name=f'NBeatsBlock_{i}'
#                                 )(residuals)

#     # Update residuals by subtracting backcast predictions
#     residuals = layers.subtract([residuals, backcast], name=f"subtract_{i}")
#     # Update forecast by adding block's forecast predictions
#     forecast = layers.add([forecast, block_forecast], name=f"add_{i}")

# # Define the N-BEATS model using the input and output layers
# N_BEATS_MODEL = tf.keras.Model(
#                                 inputs=stack_input,
#                                 outputs=forecast,
#                                 name='Model_N_BEATS'
#                                 )

# # Compile the model with Mean Absolute Error loss, Adam optimizer, and additional metrics
# N_BEATS_MODEL.compile(loss='mae', optimizer=tf.keras.optimizers.Adam(0.001), metrics=["mae", "mse"])

# # Display the summary of the model architecture
# N_BEATS_MODEL.summary()


TypeError: Cannot iterate over a Tensor with unknown first dimension.

In [3]:
window_size = 30
horizon = 1

class NBeatsBlock(tf.keras.layers.Layer):
    def __init__(self,
                input_size:int,
                theta_size:int,
                horizon:int,
                n_neurons:int,
                n_layers:int,
                **kwargs):

        super().__init__(**kwargs)

        # Initialize attributes based on input parameters
        self.input_size = input_size  # Size of the input time series
        self.theta_size = theta_size  # Size of the theta vector
        self.horizon = horizon  # Forecasting horizon
        self.n_neurons = n_neurons  # Number of neurons in each hidden layer
        self.n_layers = n_layers  # Number of hidden layers

        # Define hidden layers with ReLU activation
        self.hidden = [tf.keras.layers.Dense(n_neurons, activation="relu") for _ in range(n_layers)]

        # Define theta layer with linear activation
        self.theta_layer = tf.keras.layers.Dense(theta_size, activation="linear", name="theta")

    def call(self, inputs):
        x = inputs  # Input tensor
        # Pass input through each hidden layer sequentially
        for layer in self.hidden:
            x = layer(x)
        
        # Compute theta vector by passing output of last hidden layer through theta layer
        theta = self.theta_layer(x)

        # Split theta vector into backcast and forecast components
        backcast, forecast = theta[:, :self.input_size], theta[:, -self.horizon:]

        # Return backcast and forecast predictions
        return backcast, forecast



N_EPOCHS = 1000
N_NEURONS = 512
N_LAYERS  = 4
N_STACKS = 30

input_size = window_size * horizon
theta_size = window_size + horizon

from tensorflow.keras import layers 
tf.random.set_seed(42)

# Initialize the first N-Beats block layer
nbeat_block_layer = NBeatsBlock(
                                input_size=input_size,
                                theta_size=theta_size,
                                horizon=horizon,
                                n_neurons=N_NEURONS,
                                n_layers=N_LAYERS,
                                name='Initial_block'
                                )

# Define input layer for the stacked model
stack_input = layers.Input(shape=(input_size,), name='stack_input')

# Apply the initial N-Beats block layer to the input layer
backcast, forecast = nbeat_block_layer(stack_input)

# Calculate residuals by subtracting the backcast predictions from the input
residuals = layers.subtract([stack_input, backcast], name="subtract_00")

# Iterate over the remaining N-Beats blocks to create a stacked architecture
for i, _ in enumerate(range(N_STACKS - 1)):
    # Apply N-Beats block to residuals from the previous block
    backcast, block_forecast = NBeatsBlock(
                                input_size=input_size,
                                theta_size=theta_size,
                                horizon=horizon,
                                n_neurons=N_NEURONS,
                                n_layers=N_LAYERS,
                                name=f'NBeatsBlock_{i}'
                                )(residuals)

    # Update residuals by subtracting backcast predictions
    residuals = layers.subtract([residuals, backcast], name=f"subtract_{i}")
    # Update forecast by adding block's forecast predictions
    forecast = layers.add([forecast, block_forecast], name=f"add_{i}")

# Define the N-BEATS model using the input and output layers
N_BEATS_MODEL = tf.keras.Model(
                                inputs=stack_input,
                                outputs=forecast,
                                name='Model_N_BEATS'
                                )

# Compile the model with Mean Absolute Error loss, Adam optimizer, and additional metrics
N_BEATS_MODEL.compile(loss='mae', optimizer=tf.keras.optimizers.Adam(0.001), metrics=["mae", "mse"])

# Display the summary of the model architecture
N_BEATS_MODEL.summary()



Model: "Model_N_BEATS"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 stack_input (InputLayer)    [(None, 30)]                 0         []                            
                                                                                                  
 Initial_block (NBeatsBlock  ((None, 30),                 819743    ['stack_input[0][0]']         
 )                            (None, 1))                                                          
                                                                                                  
 subtract_00 (Subtract)      (None, 30)                   0         ['stack_input[0][0]',         
                                                                     'Initial_block[0][0]']       
                                                                                     

In [ ]:
N_BEATS_MODEL.fit(train_dataset, epochs = N_EPOCHS, callbacks = callbacks=[tf.keras.callbacks.EarlyStopping(monitor="loss", patience=200, restore_best_weights=True),
                      tf.keras.callbacks.ReduceLROnPlateau(monitor="loss", patience=100, verbose=1)])

In [5]:
saved_model_path = r"D:\Time_series_project\N_BEATS_MODEL_30_1_loss_26.28_val_loss_679.27"
loaded_model = tf.keras.models.load_model(saved_model_path)

In [108]:
def mean_absolute_scaled_error(y_true, y_pred, seasonality=1):
    mae = np.mean(np.abs(y_true - y_pred))
    naive_forecast = y_true[:-seasonality]
    naive_mae = np.mean(np.abs(naive_forecast - y_true[seasonality:]))
    mase = mae / naive_mae
    return mase

def evaluate_preds(y_true, y_pred):
  # Make sure float32 (for metric calculations)
  y_true = tf.cast(y_true, dtype=tf.float32)
  y_pred = tf.cast(y_pred, dtype=tf.float32)

  # Calculate various metrics
  mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
  mse = tf.keras.metrics.mean_squared_error(y_true, y_pred)
  rmse = tf.sqrt(mse)
  huber_loss = tf.keras.losses.Huber()(y_true, y_pred)
  mase = mean_absolute_scaled_error(y_true.numpy(), y_pred.numpy(),seasonality=1)

  # Account for different sized metrics (for longer horizons, reduce to single number)
  if mae.ndim > 0: # if mae isn't already a scalar, reduce it to one by aggregating tensors to mean
    mae = tf.reduce_mean(mae)
    mse = tf.reduce_mean(mse)
    rmse = tf.reduce_mean(rmse)
    huber_loss = tf.reduce_mean(huber_loss)
    mase = tf.reduce_mean(mase)

  print("For mae, mse, rmse and hubber : lower value is better.\nmase :- A scaled eror is >1 if the forecast is worse than the naive and <1 if the forecast is better that the naive.")
  print(f"""
    "mae": {mae.numpy()},
    "mse": {mse.numpy()},
    "rmse": {rmse.numpy()},
    "huber_loss": {huber_loss.numpy()},
    "mase": {mase}
    """)
  return {
        "mae": mae.numpy(),
        "mse": mse.numpy(),
        "rmse": rmse.numpy(),
        "huber_loss": huber_loss.numpy(),
        "mase": mase.numpy()
    }

In [109]:
forecast  = N_BEATS_MODEL.predict(feature)
preds =(forecast)
# labels.shape, feature

108/108 [==============================] - 4s 36ms/step


In [110]:
evaluate_preds(y_true= labels, y_pred= preds)

For mae, mse, rmse and hubber : lower value is better.
mase :- A scaled eror is >1 if the forecast is worse than the naive and <1 if the forecast is better that the naive.

    "mae": 3034.87255859375,
    "mse": 28722984.0,
    "rmse": 3034.87255859375,
    "huber_loss": 3034.372802734375,
    "mase": 7.742700576782227
    


{'mae': 3034.8726,
 'mse': 28722984.0,
 'rmse': 3034.8726,
 'huber_loss': 3034.3728,
 'mase': 7.7427006}

In [111]:
# model.save('model_11_multivariate_parameter_1_500.h5')